# Computational Phase Retrieval with Tensor Methods

## Device Information

In [1]:
!nvidia-smi

Tue Jul 13 04:34:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8     5W /  N/A |    343MiB /  3911MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import Required Libraries

In [2]:
import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        print("Num GPUs Available: ", len(gpus))
        for gpu in gpus:
            # Allow memory growth for the GPU.
            # Reference: https://www.tensorflow.org/guide/gpu
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized.
        print(e)

from matplotlib import pyplot as plt
plt.style.use('dark_background')
import numpy as np
import tensorly as tl
import cv2
import time
import os
from PIL import Image

import matplotlib.cm as cm
import seaborn as sns

from matplotlib import rc, rcParams
rcParams['font.family'] = 'Cubano'
rc('text', usetex=False)

Tensorflow version: 2.5.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


## GPU Benchmark

Reference: https://www.tensorflow.org/guide/gpu

In [3]:
tf.debugging.set_log_device_placement(True)

n = 5000
num_iters = 10

'''
Test with TensorFlow GPU.
'''
start_tf = time.time()

for i in range(num_iters):
    # Tensors are defaultly placed on the GPU (CPU would be considerably slower due
    # to the incurred communication cost).
    # with tf.device('/CPU:0'):
    a = tf.ones((n, n))
    b = tf.ones((n, n))

    # Run on the GPU
    c = tf.matmul(a, b)

print(f'Elapsed time with TensorFlow GPU: {time.time() - start_tf}') # about 0.02 seconds

'''
Test with Numpy.
'''
start_np = time.time()

for i in range(num_iters):
    a = np.ones((n, n))
    b = np.ones((n, n))

    c = np.dot(a, b)

print(f'Elapsed time with Numpy: {time.time() - start_np}') # about 15 seconds


Elapsed time with TensorFlow GPU: 0.5495631694793701
Elapsed time with Numpy: 16.494407176971436


## Low Rank Phase Retrieval

References:

\[1\] Namrata Vaswani, Seyedehsara Nayer, Yonina C. Eldar. *Low Rank Phase Retrieval*. https://rutgers.box.com/s/dntl0sh157p62rgi1zerdaxrqthugr32

\[2\] Namrata Vaswani. *Nonconvex Structured Phase Retrieval*. https://rutgers.box.com/s/x02w8frd1ep01cxdjlnojufa9npvstsz.

\[3\] Tamara G. Kolda, Brett W. Bader. *Tensor Decompositions and Applications*. https://rutgers.box.com/s/aq9psx3mgwhms6rrzlhn94h56c3oshox. 




### Define Data Directories

In [4]:
INPUT_DIR = './videos/' # directory of the test videos
OUTPUT_DIR = './output/' # output directory
FRAMES_DIR = './ouput/frames/' # output directory of the extracted video frames 

### Load the Test Video

In [5]:
# Read the video.
video_path = INPUT_DIR + os.listdir(INPUT_DIR)[0] # define video path
cap = cv2.VideoCapture(video_path) # read the video from path
video_name = os.listdir(INPUT_DIR)[0].split('.')[0] # get the name of the video

# Creat the folder to store the extracted frames of the video.
try:
    if not os.path.exists(FRAMES_DIR + video_name):
        os.makedirs(FRAMES_DIR + video_name)
    else:
        print('Directory already exists!')
except OSError:
    print('OS ERROR')

k = 0 # frame number, k = 0, 1, 2, ..., q - 1
X_list = []
while (True):
    # Capture the video frame-by-frame.
    # Code adopted: https://docs.opencv.org/3.4/dd/d43
    # # tutorial_py_video_display.html
    ret, frame = cap.read()

    # If the frame is read correctly the return boolean (ret) is true.
    if not ret:
        print("Cannot receive frame (probably end of stream). Exiting...")
        break
    else:
        # Convert the frame to grayscale.
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        name = FRAMES_DIR + video_name + '/frame-' + str(k) + '.jpg'
        print('DEBUG: Captured...' + name)
        cv2.imwrite(name, gray_frame)
        X_list.append(gray_frame)

        k += 1

# Release the capture when finished.
cap.release()
cv2.destroyAllWindows()

Directory already exists!
DEBUG: Captured..../ouput/frames/sara/frame-0.jpg
DEBUG: Captured..../ouput/frames/sara/frame-1.jpg
DEBUG: Captured..../ouput/frames/sara/frame-2.jpg
DEBUG: Captured..../ouput/frames/sara/frame-3.jpg
DEBUG: Captured..../ouput/frames/sara/frame-4.jpg
DEBUG: Captured..../ouput/frames/sara/frame-5.jpg
DEBUG: Captured..../ouput/frames/sara/frame-6.jpg
DEBUG: Captured..../ouput/frames/sara/frame-7.jpg
DEBUG: Captured..../ouput/frames/sara/frame-8.jpg
DEBUG: Captured..../ouput/frames/sara/frame-9.jpg
DEBUG: Captured..../ouput/frames/sara/frame-10.jpg
DEBUG: Captured..../ouput/frames/sara/frame-11.jpg
DEBUG: Captured..../ouput/frames/sara/frame-12.jpg
DEBUG: Captured..../ouput/frames/sara/frame-13.jpg
DEBUG: Captured..../ouput/frames/sara/frame-14.jpg
DEBUG: Captured..../ouput/frames/sara/frame-15.jpg
DEBUG: Captured..../ouput/frames/sara/frame-16.jpg
DEBUG: Captured..../ouput/frames/sara/frame-17.jpg
DEBUG: Captured..../ouput/frames/sara/frame-18.jpg
DEBUG: Captured

### Create the true signal tensor.

Tensors are multi-dimensional arrays with a uniform type (`dtype`). All tensors are immutable like Python numbers and strings: you can never update the contents of a tensor, only create a new one.

**Note**: In libraries like tensorflow, the rank of the tensor actually denotes the order of the tensor in our convention. We call the `rank` of a tensor in a similar manner as the rank of a matrix.

In [19]:
order_0_tensor = tf.constant(27) # int32 tensor by default
print(order_0_tensor)

order_1_tensor = tf.constant([3.0, 4.0, 5.0]) # float32 order-1 tensor (vector)
print(order_1_tensor)

order_2_tensor = tf.constant([[1, 2], # order-2 tensor (matrix) with custom dtype
                              [3, 4],
                              [5, 6]], dtype=tf.float16)
print(order_2_tensor)

order_3_tensor = tf.constant([
    [[0, 1, 2, 3, 4],
     [5, 6, 7, 8, 9]],
    [[10, 11, 12, 13, 14],
     [15, 16, 17, 18, 19]],
    [[20, 21, 22, 23, 24],
     [25, 26, 27, 28, 29]]], dtype=tf.float32)
print(order_3_tensor)

tf.Tensor(27, shape=(), dtype=int32)
tf.Tensor([3. 4. 5.], shape=(3,), dtype=float32)
tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]], shape=(3, 2), dtype=float16)
tf.Tensor(
[[[ 0.  1.  2.  3.  4.]
  [ 5.  6.  7.  8.  9.]]

 [[10. 11. 12. 13. 14.]
  [15. 16. 17. 18. 19.]]

 [[20. 21. 22. 23. 24.]
  [25. 26. 27. 28. 29.]]], shape=(3, 2, 5), dtype=float32)


The gray-scaled signal is modeled as a three-ordered tensor $\boldsymbol{\mathcal{X}} \in \mathbb{R}^{I_1 \times I_2 \times q}$, where $I_1 \times I_2$ correspond to the pixel coordinates within each frame and $q$ is the total number of frames captured.

In [22]:
X_ast = tf.constant(X_list) # true signal tensor X*

In [23]:
q, I1, I2 = X_ast.shape
print(f'The dimension of the true signal tensor: I1 x I2 x q: {I1} x {I2} x {q}')

The dimension of the true signal tensor: I1 x I2 x q: 480 x 640 x 105


print(X_ast)

### Generate Phaseless Measurements

In [25]:
tf.random.set_seed(5)
A11 = tf.random.normal([I1, I2], 0, 1, tf.float32)